In [1]:
import pickle
import numpy as np
import pandas as pd
import glob

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
device = torch.device('cpu')

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

In [2]:
from torchsummary import summary
from torch.nn import functional as F

In [3]:
from src.genetic_generator import decode_bracket

# Подготовка данных

TODO: добавить glob, переместить все дампы в папку

In [4]:
files_x = sorted(glob.glob('dumps/dump_x*.pkl'))
files_y = sorted(glob.glob('dumps/dump_y*.pkl'))
print(f'{len(files_x)} files')

810 files


In [5]:
with open(files_x[0], 'rb') as fx, open(files_y[0], 'rb') as fy:
    data_x = np.array([decode_bracket(i)[0] for i in pickle.load(fx)])
    data_y = np.array([decode_bracket(i)[0] for i in pickle.load(fy)])
    assert data_x.shape == data_y.shape
    assert len(data_x.shape) == 3

files_x.pop(0)
files_y.pop(0)

for filename_x, filename_y in zip(files_x, files_y):
    with open(filename_x, 'rb') as fx, open(filename_y, 'rb') as fy:
        ex_data_x = np.array([decode_bracket(i)[0] for i in pickle.load(fx)])
        ex_data_y = np.array([decode_bracket(i)[0] for i in pickle.load(fy)])

        assert ex_data_x.shape == ex_data_y.shape, f'Broke: {ex_data_x}, {ex_data_y}, ex_data_x.shape == ex_data_y.shape: {ex_data_x.shape == ex_data_y.shape}'
        assert len(ex_data_x.shape) == 3

        data_x = np.vstack((data_x, ex_data_x))
        data_y = np.vstack((data_y, ex_data_y))

In [6]:
data_x_r = data_x.reshape((data_x.shape[0], data_x.shape[1] * data_x.shape[2]))
data_y_r = data_y.reshape((data_y.shape[0], data_y.shape[1] * data_y.shape[2]))
swaps = []

for x, y in zip(data_x_r, data_y_r):
    swaps.append(np.where(x != y)[0])
swaps = np.array(swaps)
print(swaps)
assert swaps.shape[0] == data_x.shape[0]
assert swaps.shape[1] == 2

[[ 5  8]
 [56 69]
 [59 69]
 ...
 [34 38]
 [44 55]
 [45 55]]


In [7]:
X = torch.tensor(data_x_r.astype('float32'))
y = torch.tensor(swaps.astype('float32'))

X, X_test, y, y_test = train_test_split(X, y, test_size=0.1)
print(f'Train: {X.shape[0]}, test: {X_test.shape[0]}')

Train: 19510, test: 2168


# Полносвязная нейросеть

In [38]:
# TODO:
# test activation functions
# test architectures
# different optimizers 

In [39]:
model = nn.Sequential(
  nn.Linear(X.shape[1], X.shape[1]),
  nn.ReLU(),
  nn.Linear(X.shape[1], X.shape[1]),
  nn.ReLU(),
  nn.Linear(X.shape[1], X.shape[1]),
  nn.ReLU(),
  nn.Linear(X.shape[1], X.shape[1] // 4),  
  nn.ReLU(),
  nn.Linear(X.shape[1] // 4, 2),
  )
model.to(device)

Sequential(
  (0): Linear(in_features=70, out_features=70, bias=True)
  (1): ReLU()
  (2): Linear(in_features=70, out_features=70, bias=True)
  (3): ReLU()
  (4): Linear(in_features=70, out_features=70, bias=True)
  (5): ReLU()
  (6): Linear(in_features=70, out_features=17, bias=True)
  (7): ReLU()
  (8): Linear(in_features=17, out_features=2, bias=True)
)

In [40]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
n_epochs = 100
batch_size = 200
history = []

In [41]:
for n_epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        Xbatch = X[i:i+batch_size]
        y_pred = model(Xbatch)
        
        ybatch = y[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        history.append(loss.item())
    
    print(f'Finished epoch {n_epoch + 1}/{n_epochs}, latest loss {loss:.3f}, R^2 on test {r2_score(y_test.detach().numpy(), model(X_test).detach().floor().numpy()):.3f}, NMSE on test {mean_squared_error(y_test.detach().numpy(), model(X_test).detach().floor().numpy()):.3}')

Finished epoch 1/100, latest loss 639.026, R^2 on test -0.787, NMSE on test 7.09e+02
Finished epoch 2/100, latest loss 596.880, R^2 on test -0.655, NMSE on test 6.57e+02
Finished epoch 3/100, latest loss 562.496, R^2 on test -0.577, NMSE on test 6.26e+02
Finished epoch 4/100, latest loss 534.467, R^2 on test -0.466, NMSE on test 5.82e+02
Finished epoch 5/100, latest loss 511.640, R^2 on test -0.400, NMSE on test 5.56e+02
Finished epoch 6/100, latest loss 493.070, R^2 on test -0.339, NMSE on test 5.32e+02
Finished epoch 7/100, latest loss 477.980, R^2 on test -0.283, NMSE on test 5.09e+02
Finished epoch 8/100, latest loss 465.734, R^2 on test -0.233, NMSE on test 4.89e+02
Finished epoch 9/100, latest loss 455.811, R^2 on test -0.207, NMSE on test 4.79e+02
Finished epoch 10/100, latest loss 447.784, R^2 on test -0.164, NMSE on test 4.62e+02
Finished epoch 11/100, latest loss 441.304, R^2 on test -0.147, NMSE on test 4.55e+02
Finished epoch 12/100, latest loss 436.084, R^2 on test -0.126,

In [14]:
torch.save(model.state_dict(), 'model_big_14.pkl')

In [15]:
model.load_state_dict(torch.load('model_big_14.pkl'))
model.eval()

Sequential(
  (0): Linear(in_features=70, out_features=70, bias=True)
  (1): ReLU()
  (2): Linear(in_features=70, out_features=70, bias=True)
  (3): ReLU()
  (4): Linear(in_features=70, out_features=70, bias=True)
  (5): ReLU()
  (6): Linear(in_features=70, out_features=17, bias=True)
  (7): ReLU()
  (8): Linear(in_features=17, out_features=2, bias=True)
)

In [42]:
round(r2_score(y_test.detach().numpy(), model(X_test).detach().floor().numpy()), 3)

-0.006

# Сверточная нейронка

Данные для нее не reshape'им

In [8]:
X = torch.tensor(data_x.astype('float32'))
X = X[:, None, :, :]
y = torch.tensor(swaps.astype('float32'))

X, X_test, y, y_test = train_test_split(X, y, test_size=0.1)
print(f'Train: {X.shape[0]}, test: {X_test.shape[0]}')

Train: 19510, test: 2168


In [20]:
class ConvNetSwapAdviser(nn.Module):
    def __init__(self):
        super(ConvNetSwapAdviser, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square conv kernel
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        # self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3)
        # self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(36, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 2)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))

        # x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(-1, 36)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [21]:
model = ConvNetSwapAdviser().to(device)

summary(model, (1, 14, 5))

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
history = []
n_epochs = 15
batch_size = 30

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1             [-1, 6, 12, 3]              60
         MaxPool2d-2              [-1, 6, 6, 1]               0
            Linear-3                   [-1, 16]             592
            Linear-4                    [-1, 8]             136
            Linear-5                    [-1, 2]              18
Total params: 806
Trainable params: 806
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.01
----------------------------------------------------------------


In [22]:
for n_epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        Xbatch = X[i:i+batch_size]

        y_pred = model(Xbatch)
        
        ybatch = y[i:i+batch_size]

        loss = criterion(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        history.append(loss.item())
    

    print(f'''Finished epoch {n_epoch + 1}/{n_epochs}, 
          latest loss {loss:.3f}, 
          R^2 on test {r2_score(y_test.detach().numpy(), model(X_test).detach().floor().numpy()):.3f}, 
          NMSE on test {mean_squared_error(y_test.detach().numpy(), model(X_test).detach().floor().numpy()):.3}''')

Finished epoch 1/15, 
          latest loss 46.873, 
          R^2 on test -3.072, 
          NMSE on test 1.66e+03
Finished epoch 2/15, 
          latest loss 46.873, 
          R^2 on test -3.072, 
          NMSE on test 1.66e+03
Finished epoch 3/15, 
          latest loss 46.873, 
          R^2 on test -3.072, 
          NMSE on test 1.66e+03
Finished epoch 4/15, 
          latest loss 46.877, 
          R^2 on test -3.074, 
          NMSE on test 1.66e+03
Finished epoch 5/15, 
          latest loss 46.873, 
          R^2 on test -3.072, 
          NMSE on test 1.66e+03
Finished epoch 6/15, 
          latest loss 46.873, 
          R^2 on test -3.072, 
          NMSE on test 1.66e+03
Finished epoch 7/15, 
          latest loss 46.873, 
          R^2 on test -3.072, 
          NMSE on test 1.66e+03
Finished epoch 8/15, 
          latest loss 46.873, 
          R^2 on test -3.072, 
          NMSE on test 1.66e+03
Finished epoch 9/15, 
          latest loss 46.873, 
          R^2 on te